## Principe du transfert de style :

 Le transfert de style est un processus permettant de transferer le style d'une image à une image dont l'on souhaite garder le contenue. 

# Architecture

On considére deux images de même taille : une correspondant à l'image contenue notée $\textbf{ImgC}$ et une correspondant à l'image style notée $\textbf{ImgS}$.  Le transfert de style repose sur un mécanisme qui comprend un encodeur, ADAIN et un décodeur. 

## 1. Encodeur 

L'encodeur va permettre d'extraire les $\textbf{features}$ de l'image de contenue $ImgC$  d'une part et les $\textbf{features}$ de l'image de style $ImgS$ d'autre part, que l'on note respectivement $\text{f(ImgC)}$ et $\text{f(ImgS)}$.  Cette encodeur est composé de 13 couches de réseaux de neuronnes dont 10 couches de convolutions et 3 couche de Maxpoolings. Considérons une image d'entrée de dimension (h,w,c), alors on décrit l'encodeur par le tableau suivant : 

![Texte alternatif de l'image](Encodeur.png)



où h, w et c sont respectivement la hauteur, la largeur et le nombre de canaux de l'image en entrée. De plus, les dimensions des cartes fonctionnelles intermédiaires sont données par les formules suivantes:   $w_1$=$[\frac{w}{2}-1]+1$ , $h_1$= $[\frac{h}{2}-1]+1$ , $w_2$=$[\frac{w_1}{2}-1]+1$, $h_2$=$[\frac{h_1}{2}-1]+1$, $w_3$=$[\frac{w_2}{2}-1]+1$ et $h_3$=$[\frac{h_2}{2}-1]+1$

$\textbf{Remarque} :$

L’encodeur transforme l’entrée en une représentation dans un espace de dimension plus faible appelé espace latent. L’encodeur compresse donc l’entrée dans une représentation moins coûteuse. 

## 2. ADAIN 

Après l’encodeur, nous connaissons la taille de la carte de fonctionnalités : pour une image d’entrée (w,h,C), nous obtenons une carte de fonctionnalités de dimension ($w_3$, $h_3$, 512) pour l'image de style et de contenue.  L'instance de normalisation adaptative opère en normalisant la carte de fonctionnalités de l'image contenue, représentée par $x = \text{f(ImgC)}$, en se basant sur les statistiques obtenues à partir de la carte de fonctionnalités de l'image style, notée $y = \text{f(ImgS)}$. Nous avons illustrer le mécanisme par le schéma suivant :

![Texte alternatif de l'image](ADAIN.png)


La première étape de ce processus implique le calcul de la moyenne des valeurs des pixels de chaque canal sur toute la largeur et la hauteur du tenseur d'entrée, ainsi que l'écart-type des valeurs des pixels de chaque canal sur l'ensemble de ces dimensions. En résultat, nous obtenons des vecteurs moyennes $\mu_c$ et $\mu_s$ pour chaque canal de la carte de fonctionnalités de contenu et de style, accompagnés de leurs écart-types respectifs notés  $\sigma_c$ et $\sigma_s$, tous ces vecteurs étant de dimension 512.

La seconde étape consiste à normalisé chacun des pixels de la carte de fonctionnalité de contenues par la moyenne $\mu_c^c$ et l'écart-type $\sigma_c^c$ de tout valeurs des pixels qui appartiennent au même canal. Cette valeur va être alignée au valeur de la carte fonctionnalité de style en y multipliant l'écart-type $\sigma_s^c$ de tout valeurs des pixels qui appartiennent au même canal dans la carte de fonctionnalité style et en y ajoutant la moyenne $\mu_s^c$.

A la fin de ces étapes, nous obtenons donc la carte de fonctionalité cible que l'on note t= $ADAIN(x,y)$.

## Extension de Instance normalisation, 

## 2. Décodeur 

A partir du $\textbf{feature de cible}$, c'est-à-dire les features obtenues par la méthode ADAIN, le décodeur va essayer de reconstruire l'image contenue stylisé. Le décodeur est composé de 12 couches de réseaux de neuronnes dont 10 couches de convolutions et  de Upsampling.  

![Texte alternatif de l'image](decodeur.png)

$\textbf{Remarque} $ : 
    On remarque la présence de 3 couche UPSAMPLING permettant de redimensionner ou agrandir le tenseur d'entrée d’un facteur 2, ‘nearest’ indique une interpolation par le plus proche voisin. Cette méthode attribue la valeur du pixel d'entrée le plus proche au pixel correspondant en sortie, en faisant une méthode d'interpolation simple et rapide.

#### Nombre total de paramétres à estimer :

5 240 704 + 3 * c paramétres à estimer 


## Entrainement 

Le modéle est entrainé sur 80 000 images, on utilise l'optimiseur ADAM et le taille de batch est égale à 8.

## Perte liée au contenue

   Cette perte permet de quantifier la perte liées au contenue en effet l’encodeur va permettre d’extraire les features des images de style $f(ImgS)$ et de contenue $f(ImgC)$. Ces features vont être transmise à la couche $ADAIN()$ qui aligne la moyenne et la variance des cartes de fonctionnalités de contenu sur celles de les cartes de caractéristiques de style, produisant les caractéristiques cibles que l'on note t= $ADAIN(f(ImgS), f(ImgC))$. 

![Texte alternatif de l'image](architecture.jpg)

 Ensuite cette carte de fonctionnalités va être reconstruite par un décodeur afin d’avoir une image ayant le contenue et style souhaité g(t) . Ensuite, on va extraire la carte fonctionnelle de cette image dont le contenue à été stylisé, on passe par un encodeur : on obtient les caractéristique de l’image reconstruite ayant le contenue et style souhaité f(g(t)). 


$$ L_c = || f(g(t)) -t||_2$$

La perte du contenue correspond à la différence euclidienne entre les caractéristiques de fonctionnalité à la sortie de l’ADAIN() et celle extraite de l’image reconstruite ayant le contenue et style souhaité, dont on vérifie bien la compatibilité au niveau des dimensions. 

### PRÉ-entrainement ?

## Perte liée au style

## Limitation